In [1]:
import torch
from torch import nn
from d2l import torch as d2l
import time

In [2]:
torch.cuda.is_available()

True

1. Try a larger computation task, such as the multiplication of large matrices, and see the difference in speed between the CPU and GPU. What about a task with a small number of calculations?

In [16]:
def matMul(dimension, device, n):
    X = torch.randn(dimension, dimension, device=device)
    Y = torch.randn(dimension, dimension, device=device)
    time1 = time.time()
    for i in range(n):
        Z = torch.matmul(X, Y)
    time2 = time.time()
    return time2-time1

In [18]:
print("cpu 10000x10000 ", matMul(5000, 'cpu', 5))
print("gpu 10000x10000 ", matMul(5000, 'cuda', 5))
print("cpu 100x100 ", matMul(500, 'cpu', 5))
print("gpu 100x100 ", matMul(500, 'cuda', 5))

cpu 10000x10000  1.7620086669921875
gpu 10000x10000  0.0
cpu 100x100  0.001995086669921875
gpu 100x100  0.010004758834838867


When doing a larger computation task, using GPU is much faster than CPU. When doing a small number of calculations,the two methods take similar time.

2. How should we read and write model parameters on the GPU?

In [ ]:
PATH = "model.pt"

model = d2l.LinearRegression()

# Save
torch.save(model.state_dict(), PATH)

# Load
device = torch.device("cuda")
model = d2l.LinearRegression()
model.load_state_dict(torch.load(PATH))
model.to(device)

3. Measure the time it takes to compute 1000 matrix-matrix multiplications of 100 × 100 matrices and log the Frobenius norm of the output matrix one result at a time vs. keeping a log on the GPU and transferring only the final result.

In [40]:
A = torch.randn(100, 100, device='cuda')
B = torch.randn(100, 100, device='cuda')
time1 = time.time()
logs = torch.tensor((1,1), device='cpu')
for i in range(3000):
    result = torch.matmul(A, B).norm()
    torch.cat((logs, result.reshape(1).to('cpu')))

time2 = time.time()
print(time2-time1)

0.362776517868042


In [41]:
A = torch.randn(100, 100, device='cuda')
B = torch.randn(100, 100, device='cuda')
time1 = time.time()
logs = torch.tensor((1,1), device='cuda')
for i in range(3000):
    result = torch.matmul(A, B).norm()
    torch.cat((logs, result.reshape(1)))
logs.to('cpu')

time2 = time.time()
print(time2-time1)

0.14913058280944824


The latter one is faster.